# Seasonal Decompose Feature Extractor

* Seasonal Strength, Trend Strength, Residual Variability

* Source Link via Rob Hyndman: 
    * https://otexts.com/fpp2/seasonal-strength.html

In [7]:
import polars as pl

from polars_ts.decomposition.seasonal_decompose_features import seasonal_decompose_features

In [8]:
df = (
    pl.scan_parquet("https://datasets-nixtla.s3.amazonaws.com/m5_y.parquet")
    .filter(pl.col("unique_id").is_in(pl.col("unique_id").unique().sample(500)))
    .collect()
)

/var/folders/gw/jz9ttp7x4gd8t_h9_bscxzc40000gn/T/ipykernel_11574/1192745697.py:3: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  .filter(pl.col('unique_id').is_in(pl.col('unique_id').unique().sample(500))).collect()


# Plot Results

* Now we see a relative measure of Seasonality/Trend Over All Series.

In [9]:
simple_decompose_features = seasonal_decompose_features(
    df=df, id_col="unique_id", time_col="ds", target_col="y", ts_freq=365, mode="simple"
)

mstl_decompose_features = seasonal_decompose_features(
    df=df, id_col="unique_id", time_col="ds", target_col="y", ts_freq=365, mode="mstl", seasonal_freqs=[7, 28, 90]
)

# Plot Naive Decomposition Features

* 24 period seasonality 
* Simple Moving Average Trend (Naive)

In [10]:
seas = simple_decompose_features["seasonal_strength"].hvplot.hist(
    alpha=0.6, bins=50, title="Hyndman's Seasonal and Trend Strength"
)

trend = simple_decompose_features["trend_strength"].hvplot.hist(alpha=0.6, bins=50)

seas * trend

:Overlay
   .Histogram.I  :Histogram   [seasonal_strength]   (Count)
   .Histogram.II :Histogram   [trend_strength]   (Count)

# Plot MSTL Features 
    * Multiple Seasonal Components can be examined 
    * LOESS Trend as Opposed to MA Trend

In [11]:
# Assuming `mstl_feats` is a DataFrame containing the columns
# 'seasonal24_strength', 'seasonal48_strength', 'seasonal167_strength'

# Customizing the plot
hist_plot = mstl_decompose_features.hvplot.hist(
    y=["seasonal7_strength", "seasonal28_strength", "seasonal90_strength"],
    title="Normalized MSTL Components of All Series",
    color=["#1f77b4", "#ff7f0e", "#2ca02c"],  # Custom color palette
    alpha=0.6,  # Transparency for the bars
    legend="top_left",  # Place legend in the top-left corner
    width=800,  # Set a custom width for the plot
    height=500,  # Set a custom height for the plot
    bins=50,  # Number of bins
    xlabel="Seasonal Strength S(t)",  # Label for the x-axis
    ylabel="Frequency",  # Label for the y-axis
    grid=True,  # Enable gridlines
    invert_axes=False,  # Standard axes (not inverted)
)

# plot trend strength
trend_plot = mstl_decompose_features.hvplot.hist(
    y=["trend_strength"],
    title="Normalized LOESS Component Of All Series",
    color=["#1f77b4", "#ff7f0e", "#2ca02c"],  # Custom color palette
    alpha=0.6,  # Transparency for the bars
    legend="top_left",  # Place legend in the top-left corner
    width=800,  # Set a custom width for the plot
    height=500,  # Set a custom height for the plot
    bins=50,  # Number of bins
    xlabel="Trend Strength T(t)",  # Label for the x-axis
    ylabel="Frequency",  # Label for the y-axis
    grid=True,  # Enable gridlines
    invert_axes=False,  # Standard axes (not inverted)
)

# Display the plot
hist_plot + trend_plot

:Layout
   .NdOverlay.I  :NdOverlay   [Variable]
      :Histogram   [seasonal90_strength]   (Count)
   .NdOverlay.II :NdOverlay   [Variable]
      :Histogram   [trend_strength]   (Count)